## Adding geospatial data to main df

In [10]:
# Importing libraries
import geopandas as gpd
import pandas as pd

# Opening datasets
neighbourhoods = gpd.read_file('Data/limites-administratives-des-communes-en-region-de-bruxelles-capitale.geojson')
listings = pd.read_excel('Data/listings.xlsx')

In [12]:
listings.columns.tolist()

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [16]:
airbnb_geo = gpd.GeoDataFrame(
    listings,
    geometry = gpd.points_from_xy(listings['longitude'], listings['latitude']),
    crs='EPSG:4326'  # Standard lat/long coordinate system
)

In [20]:
# Make sure both use the same coordinate system
neighbourhoods = neighbourhoods.to_crs('EPSG:4326')

In [26]:
final_df = gpd.sjoin(airbnb_geo, neighbourhoods, how='left', predicate='within')
final_df.to_csv('listings_with_neighborhoods.csv', index=False)